In [1]:
from sae_lens import HookedSAETransformer, SAE, SAEConfig
from gemma_utils import get_gemma_2_config, gemma_2_sae_loader, get_all_string_min_l0_resid_gemma
import numpy as np
import torch
import tqdm
import einops
import re
from jaxtyping import Int, Float
from typing import List, Optional, Any
from torch import Tensor
import json
import os
from torch.utils.data import Dataset, DataLoader
import random
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
import random
from transformer_lens.utils import get_act_name

In [2]:
torch.cuda.empty_cache()
np.random.seed(0)
random.seed(0)
torch.random.manual_seed(0)

In [3]:
model = HookedSAETransformer.from_pretrained("google/gemma-2-2b-it", device = "cpu")
#model.to("cuda:0")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model google/gemma-2-2b-it into HookedTransformer


In [4]:
from datasets import load_dataset

ds = load_dataset("santhoshmlops/Skai_Gemma_Instruct_ChatTemplate",split = "train")
text = ds[:1000]['text']
tokenized_ds = model.tokenizer(text, return_tensors = "pt", padding = True)
pairs_length = tokenized_ds['attention_mask'].sum(dim = -1).numpy()
indices = torch.where(torch.tensor(np.logical_and(pairs_length>50, pairs_length<200)))
text_array = np.array(text)[indices]
samp_ind = torch.randperm(len(text_array))[:300]
final_text = text_array[samp_ind].tolist()

In [7]:

text = final_text[0]
text = text.split("<statr_of_turn>model")[0]+"<start_of_turn>model"
tokens = model.to_tokens(text, prepend_bos = False)
model.to("cuda:0")
with torch.no_grad():
        out = model.generate(
            tokens,
            max_new_tokens = 400,
            temperature = 0.7,
            top_p = 0.9,
            eos_token_id=107,
            stop_at_eos=True,
            )
       

Moving model to device:  cuda:0


  0%|          | 0/400 [00:00<?, ?it/s]

In [52]:
string = "<bos><start_of_turn>user Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n Sally has 2 chickens, 3 cows and 2 motorcycles. Doug has 6 monkeys, 12 pigs and 2 tractors. Amy has 1 car, 10 cows and 6 ducks. How many farm animals do Sally, Doug and Amy have in total? <end_of_turn>\n<start_of_turn>model \n Chickens, cows, ducks and pigs are commonly considered to be farm animals because they are commonly raised in farms and are grown for human consumption. Meanwhile, monkeys are not considered to be farm animals because they often grow up in the wild and are not often consumed by humans. Cars, motorcycles and tractors are not living things so they are not farm animals. Sally, Doug and Amy have 2 + 3 + 12 + 10 + 6 = 33 farm animals in total. <end_of_turn><start_of_turn>model \n\nHere's how to calculate the total number of farm animals:\n\n* **Sally:** 2 chickens + 3 cows = 5\n* **Doug:** 6 monkeys + 12 pigs + 2 tractors = 20\n* **Amy:** 1 car + 10 cows + 6 ducks = 17\n\n**Total:** 5 + 20 + 17 = **42**\n\nSally, Doug, and Amy have a total of 42 farm animals. \n<end_of_turn>"
tokens = model.to_tokens(string,prepend_bos = False)


In [42]:
tokens

tensor([[     2,    106,   1645,  30641,    603,    671,  14239,    674,  19306,
            476,   6911, 235265,  15615,    476,   3590,    674,  47258,  56382,
            573,   3853, 235265,    109,  47239,    919, 235248, 235284,  53190,
         235269, 235248, 235304,  35768,    578, 235248, 235284,  83699, 235265,
          17487,    919, 235248, 235318,  63495, 235269, 235248, 235274, 235284,
          41612,    578, 235248, 235284, 108160, 235265,  27161,    919, 235248,
         235274,   1226, 235269, 235248, 235274, 235276,  35768,    578, 235248,
         235318,  64322, 235265,   2250,   1767,   7091,   8398,    749,  47239,
         235269,  17487,    578,  27161,    791,    575,   3051, 235336, 235248,
            107,    108,    106,   2516, 235248,    108, 227128, 235269,  35768,
         235269,  64322,    578,  41612,    708,  17203,   5604,    577,    614,
           7091,   8398,   1861,    984,    708,  17203,   9244,    575,  26511,
            578,    708,  13

In [28]:

from attribution_utils import calculate_feature_attribution 
from torch.nn.functional import log_softmax

In [43]:

def metric_fn(logits: torch.Tensor) -> torch.Tensor:
    log_probs = log_softmax(logits, dim = -1)
    #x = - torch.gather(log_probs[:,:-1,:],-1, tokens[:,1:].unsqueeze(-1))
    x = log_probs[0,-2,107]
    return x

In [14]:

full_strings= get_all_string_min_l0_resid_gemma()
saes_dict = {}
with torch.no_grad():
    for layer in tqdm.tqdm([22,23]):

        repo_id = "google/gemma-scope-2b-pt-res"
        folder_name = full_strings[layer]

        config = get_gemma_2_config(repo_id, folder_name)
        cfg, state_dict, log_spar = gemma_2_sae_loader(repo_id, folder_name)
        sae_cfg = SAEConfig.from_dict(cfg)
        sae = SAE(sae_cfg)
        sae.load_state_dict(state_dict)
        #sae.to("cuda:0")
        saes_dict[sae.cfg.hook_name] = sae

/root/GemmaExploration/gemma_utils.py:158: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  resid_dict = df[df['release'] == "gemma-scope-2b-pt-res"]['saes_map'][0]
100%|██████████| 2/2 [00:01<00:00,  1.19it/s]


In [44]:

feature_attribution_df = calculate_feature_attribution(
    model = model,
    input = tokens,
    metric_fn = metric_fn,
    include_saes=saes_dict,
    include_error_term=True,
    return_logits=True,
)

In [56]:
toks = model.to_str_tokens(tokens)
unique_tokens = [f"{i}/{t}" for i, t in enumerate(toks)]
import plotly.express as px

px.bar(x = unique_tokens,
       y = feature_attribution_df.sae_feature_attributions[sae.cfg.hook_name][0].sum(-1).detach().cpu().numpy())

In [54]:
toks

['<bos>',
 '<start_of_turn>',
 'user',
 ' Below',
 ' is',
 ' an',
 ' instruction',
 ' that',
 ' describes',
 ' a',
 ' task',
 '.',
 ' Write',
 ' a',
 ' response',
 ' that',
 ' appropriately',
 ' completes',
 ' the',
 ' request',
 '.',
 '\n\n',
 ' Sally',
 ' has',
 ' ',
 '2',
 ' chickens',
 ',',
 ' ',
 '3',
 ' cows',
 ' and',
 ' ',
 '2',
 ' motorcycles',
 '.',
 ' Doug',
 ' has',
 ' ',
 '6',
 ' monkeys',
 ',',
 ' ',
 '1',
 '2',
 ' pigs',
 ' and',
 ' ',
 '2',
 ' tractors',
 '.',
 ' Amy',
 ' has',
 ' ',
 '1',
 ' car',
 ',',
 ' ',
 '1',
 '0',
 ' cows',
 ' and',
 ' ',
 '6',
 ' ducks',
 '.',
 ' How',
 ' many',
 ' farm',
 ' animals',
 ' do',
 ' Sally',
 ',',
 ' Doug',
 ' and',
 ' Amy',
 ' have',
 ' in',
 ' total',
 '?',
 ' ',
 '<end_of_turn>',
 '\n',
 '<start_of_turn>',
 'model',
 ' ',
 '\n',
 ' Chickens',
 ',',
 ' cows',
 ',',
 ' ducks',
 ' and',
 ' pigs',
 ' are',
 ' commonly',
 ' considered',
 ' to',
 ' be',
 ' farm',
 ' animals',
 ' because',
 ' they',
 ' are',
 ' commonly',
 ' raised',
 '

In [50]:
unique_tokens

['0/<bos>',
 '1/<start_of_turn>',
 '2/user',
 '3/ Below',
 '4/ is',
 '5/ an',
 '6/ instruction',
 '7/ that',
 '8/ describes',
 '9/ a',
 '10/ task',
 '11/.',
 '12/ Write',
 '13/ a',
 '14/ response',
 '15/ that',
 '16/ appropriately',
 '17/ completes',
 '18/ the',
 '19/ request',
 '20/.',
 '21/\n\n',
 '22/ Sally',
 '23/ has',
 '24/ ',
 '25/2',
 '26/ chickens',
 '27/,',
 '28/ ',
 '29/3',
 '30/ cows',
 '31/ and',
 '32/ ',
 '33/2',
 '34/ motorcycles',
 '35/.',
 '36/ Doug',
 '37/ has',
 '38/ ',
 '39/6',
 '40/ monkeys',
 '41/,',
 '42/ ',
 '43/1',
 '44/2',
 '45/ pigs',
 '46/ and',
 '47/ ',
 '48/2',
 '49/ tractors',
 '50/.',
 '51/ Amy',
 '52/ has',
 '53/ ',
 '54/1',
 '55/ car',
 '56/,',
 '57/ ',
 '58/1',
 '59/0',
 '60/ cows',
 '61/ and',
 '62/ ',
 '63/6',
 '64/ ducks',
 '65/.',
 '66/ How',
 '67/ many',
 '68/ farm',
 '69/ animals',
 '70/ do',
 '71/ Sally',
 '72/,',
 '73/ Doug',
 '74/ and',
 '75/ Amy',
 '76/ have',
 '77/ in',
 '78/ total',
 '79/?',
 '80/ ',
 '81/<end_of_turn>',
 '82/\n',
 '83/<st

In [33]:
feature_attribution_df.sae_feature_attributions[sae.cfg.hook_name][0].sum(-1).detach().cpu().numpy().shape

(304,)

In [36]:
import pandas as pd
feature_attribution_df.sae_feature_attributions[sae.cfg.hook_name][0].sum(-1).detach().cpu().numpy()
def convert_sparse_feature_to_long_df(sparse_tensor: torch.Tensor) -> pd.DataFrame:
    """
    Convert a sparse tensor to a long format pandas DataFrame.
    """
    df = pd.DataFrame(sparse_tensor.detach().cpu().numpy())
    df_long = df.melt(ignore_index=False, var_name='column', value_name='value')
    df_long.columns = ["feature", "attribution"]
    df_long_nonzero = df_long[df_long['attribution'] != 0]
    df_long_nonzero = df_long_nonzero.reset_index().rename(columns={'index': 'position'})
    return df_long_nonzero

df_long_nonzero = convert_sparse_feature_to_long_df(feature_attribution_df.sae_feature_attributions[sae.cfg.hook_name][0])
df_long_nonzero.sort_values("attribution", ascending=False)





,position,feature,attribution
34983,303,8751,2.767849
49340,303,12545,1.666400
25633,0,6460,0.999610
25634,1,6460,0.996967
10883,303,2674,0.974223
...,...,...,...
7651,303,1855,-1.845463
61744,303,15775,-2.050848
11106,303,2759,-2.273624
49586,303,12639,-3.842000
